In [ ]:
from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr
from pydantic import BaseModel

/home/ypatios/Desktop/AI-Agents/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv(override=True)
assistant = OpenAI()

In [3]:
# read the pdf file
reader = PdfReader("data/chaniotakos.pdf")
bio = ""
for page in reader.pages:
    bio += page.extract_text()
print(bio)

Ypatios Chaniotakos 
+30 6987473608 | ypatioschaniotakos@gmail.com | Athens, Greece 
github.com/YpatiosCh | linkedin.com/in/ypatios-chaniotakos-b2588b266 
Professional Summary 
Results-driven professional transitioning to software development with 12 years of 
high-stakes customer service experience. Combining technical acumen with proven 
expertise in team leadership and problem resolution. Known for quick learning, 
attention to detail, and thriving in fast-paced environments. 
Technical Skills & Competencies 
Languages & Tools: JavaScript, Golang, HTML/CSS, Rust (Learning) | Git, Docker, VS 
Code | MySQL, SQLite, MongoDB 
Core Strengths: 
• Technical: Algorithm optimization, clean code practices, system architecture, 
version control 
• Development: Peer programming, test-driven development, continuous learning 
• Leadership: Team coordination, knowledge sharing, project management 
Education 
Full-Stack Software Engineering Program | Zone01 Athens | 05/2024 - Present 
 Elite projec

In [4]:
# read summary.txt
with open("data/summary.txt", "r") as file:
    summary = file.read()
print(summary)

My name is Ypatios Chaniotakos. Currently I work as a waiter (but not for too long). My passion for programming has set the stage for me to pursue a carreer as a software engineer.
Artificial Intelligence and Blockchain are the most exciting fields of study and I really hope I can dive deep into these concepts professionally.
My hobbies are running, lifting some weights, studying the markets for some trading here and there and also spending time with my loved ones.
I love dad jokes, and I also love responding with a pun.


In [5]:
name = "Ypatios Chaniotakos"

In [6]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{bio}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [ ]:
# create the evaluation schema
class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [9]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{bio}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [10]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [11]:
evaluator = OpenAI()

In [12]:
def evaluate(reply, message, history) -> Evaluation:
    messages = [{"role": "system", "content": evaluator_system_prompt}]
    user_prompt = evaluator_user_prompt(reply, message, history)
    messages.append({"role": "user", "content": user_prompt})
    response = evaluator.chat.completions.parse(
        model="gpt-4o-mini",
        messages=messages,
        response_format=Evaluation, 
        temperature=0.1
    )
    return response.choices[0].message.parsed


In [13]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = assistant.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [14]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
        
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = assistant.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply = response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [ ]:
# start chatbot
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


Passed evaluation - returning reply
Failed evaluation - retrying
The Agent's response is not acceptable. It uses a playful form of Pig Latin, which may not be appropriate in a professional context, especially when responding to a potential client or employer. The response should maintain a professional tone and clearly ask for clarification without using informal or playful language. A better response would acknowledge the user's question and politely request clarification in standard English.
